## round(100*(bike.isnull().sum()/len(bike)), 2).sort_values(ascending=False)
isnull().sum()을 통해 결측값을 확인하는 것에서 그치지 않고 len(bike)로 나눠서 만약 결측값이 있다면, 결측값이 있는 column에서 결측값이 전체 데이터 중에 몇%인지를 확인할 수 있다.

## bike_dup = bike.copy()
## bike_dup.drop_duplicates(subset=None, inplace=True)
중복 값의 유무를 확인할 때는 해당 데이터프레임을 복사하고, 복사한 것에서 중복된 값을 제거한 후 기존의 데이터프레임과 제거한 데이터프레임의 모양을 비교한다. 모양이 같으면 기존 데이터프레임에 중복 값이 없다는 것을 의미한다. 

## bike_new = pd.get_dummies(bike_new, drop_first=True,dtype=int)
get_dummies를 할 때, 일반적으론 category형 데이터가 0과 1로 표현이 되지만 True, False의 bool 형식으로 데이터가 변하는 경우가 존재하기 때문에 dtype=int를 사용해주면 0과 1로 잘 표현이 된다. <br>
one-hot vector로 표현하는 경우, 3개의 범주에 대해서 A는 100, B는 010, C는 001으로 변하면, C=1-(A+B)로 표현이 가능해서 **다중 공선성**문제가 발생한다. 따라서 drop_first=True를 통해 첫 번째 더미 변수를 제거해서 다중 공선성 문제를 해결할 수 있다.

## 범주형 변수 시각화
범주형 변수들을 boxplot으로 시각화했을 때 boxplot의 box의 크기가 크다면 데이터의 변동성이 큰 것을 의미한다. <br>
box에 나타나는 중앙값의 위치에 따라,
- 중앙값이 상자 중간: 데이터가 대칭적으로 분포
- 중앙값이 상자 하단: 오른쪽으로 치우친 분포(positive skew）
- 중앙값이 상자 상단: 왼쪽으로 치우친 분포(negative skew)

## correlation matrix
변수들의 상관관계는 heatmap을 통해 시각적으로 확인할 수 있다. 상관계수가 -1이나 1에 가까워질수록 두 변수의 상관관계는 높다고 판단한다. <br>
sns.heatmap(bike_new.corr(), annot = True, cmap="RdBu") 에서 bike_new.corr()라고 하는 대신, 상관계수에 **절댓값**을 씌워서 **np.abs(bike_new.corr())** 를 하면 단순히 heatmap에서 진한 색상으로 표현될수록 상관계수 값이 높다고 판단할 수 있다. <br>
자전거 대여량 데이터처럼 변수들이 많은 경우, correlation matrix 전체를 시각화하면 복잡할 수 있기 때문에, heatmap( )에 **mask=np.triu(bike_new.corr())** 을 추가하면 correlation matrix에서 오른쪽 위 삼각형 절반의 데이터는 표시하지 않는다. correlation matrix는 symmetric 성질이 있기 때문에 위의 절반을 제거해도 관계들을 파악하는데 문제가 없다.  

## RFE(Recursive Feature Elimination)
RFE는 반복적으로 특성을 제거하면서 최적의 특성 집합을 찾는 방법이다. <BR>
전체 특성으로 먼저 학습을 한 뒤 각 특성의 중요도를 평가하고, 중요도가 가장 낮은 특성을 하나 제거하고 나머지 특성들로 모델을 다시 학습시킨다. 해당 과정은 **rfe = RFE(estimator=lm, n_features_to_select=15)** 에서 n_features_to_select로 설정한 개수만큼의 특성이 남을 때까지 반복한다. <br>
**list(zip(X_train.columns,rfe.support_,rfe.ranking_))** 에서 rfe.support_가 True인 column들이 최종적으로 선택된 column들을 의미한다. rfe.ranking_은 얼마나 중요한 순서로 평가됐는지를 나타낸다. 즉 rfe.ranking_이 1이면 가장 중요한 특성이라는 것을 의미한다.  <br>
RFE로 선택된 특성들만을 사용하여 모델을 학습하는것도 가능하다. 

## VIF check
RFE로 선택된 특성들을 사용하여 모델을 학습한다. 이때 VIF 점수가 높은 특성은 다중 공선성의 위험이 높으므로 제거해준다. 단, VIF 점수가 높더라도 종속변수를 예측하는데 많은 영향을 줄 것으로 판단되는 변수는 제거하는 것을 보류한다. 

## p-value
모델을 학습시킨뒤 summary()를 확인해서 p-value가 높은 변수를 제거한다. p-value가 높다는 것은 그 변수가 종속변수에 유의미한 영향을 미치지 않는다고 간주하기 때문에 제거해준다. 

## 모델 해석
F-statistic 값이 1보다 크고(233.8) p-value가 0.000이므로 전체 모형이 유의하다는 것을 확인할 수 있다.  

## 오차항의 평균이 0이라는 가정
오차항의 평균이 0이라는 것은 모델의 예측값과 실제값 사이의 차이가 무작위적이라는 것을 의미한다. 즉 회귀모델이 데이터의 패턴을 잘 설명하고 있고, 특정 방향으로 편향되지 않았다는 것을 의미한다. <br>
오차항의 평균이 0이 아니라면 모델이 특정 구간에서만 정확하고 다른 구간에서는 크게 빗나가는 경우일 수 있다. 오차항의 평균이 양수라면 **실제값에 비해 일관되게 낮은 값을 예측**하고 있음을, 평균이 음수라면 **일관되게 높은 값을 예측**하고 있음을 나타낸다. 

## R^2값과 조정된 R^2값
R^2값은 회귀모델이 종속변수의 변동성을 얼마나 잘 설명하는지를 나타내는 지표이다. 그러나 새로운 독립변수를 추가할수록 모델이 설명할 수 있는 변동성이 늘어나기 때문에, 독립변수를 추가할수록 R^2값이 커진다는 한계가 있다. <BR>
조정된 R^2값은 독립변수의 개수를 고려해서 R^2값을 조정한 것이다. 즉, 새로운 변수가 추가되면, 그 변수가 유의미할때만 조정된 R^2값이 증가한다. 따라서 모델의 설명력을 더 신뢰할 수 있게 된다.  